Import packages

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

Set global variables

In [ ]:
OUTPUT_PATH = '../data/postprocessing/240214_benchmark/'

CO2_PRICE = 95.98
HEAT_PRICE = 0
H2_PRICE = 81.01

Read output data

In [ ]:
output_df = pd.read_csv(OUTPUT_PATH + 's3_output.csv', index_col=0)

Prepare class for Assets

In [ ]:
class Assets:
    """Class for evaluating asset output data."""

    def __init__(self, input_data: pd.DataFrame) -> None:
        self.ids = []
        self.input_data = input_data
        self.asset_data = {}
        self.postprocess_data = {}
        self.objective_value = None

        self.get_asset_ids()
        self.get_asset_data()
        self.calc_postprocess_data()
        self.get_objective_value_from_log()


    def get_asset_ids(self):
        """Get all unique asset names from the input data."""
        ids_set = set()

        for column_name in self.input_data.columns:
            if not '.' in column_name:
                continue
            key = re.sub('\..*$', '', column_name)      # Replace every character after a '.'.
            key = key.replace('_expanded', '')          # Replace additional characters.
            ids_set.add(key)                            # Remove duplicate keys.
        
        for id in ids_set:
            self.ids.append(id)
        
        self.ids.sort()

        if not self.ids:
            raise ValueError("""Found no asset ids in input data.""")


    def get_asset_data(self):
        """Get the specific asset data from the input data."""

        for id in self.ids:
            asset_df = pd.DataFrame()

            for column_name in self.input_data.columns:
                if id in column_name:
                    key_name = re.sub('^.*\.', '', column_name)
                    asset_df[key_name] = self.input_data[column_name]
            
            self.asset_data[id] = asset_df


    def calc_postprocess_data(self):
        """Calculates the sum of all columns in the asset data."""

        for key in self.asset_data:
            asset_dict = {}
            for column in self.asset_data[key].columns:
                column_sum = self.asset_data[key][column].sum()
                asset_dict[column] = column_sum
                self.postprocess_data[key] = asset_dict
    
    
    def get_objective_value_from_log(self):
        """Gets the objective value from the solver log file."""

        with open(OUTPUT_PATH + 's3_solver.log', 'r') as log_file:
            for line in log_file:
                if 'Best objective' in line:
                    line = line.replace('Best objective ', '')
                    line = re.sub(',.*$', '', line)
                    self.objective_value = round(float(line), 1)
                    

    def check_objective_value(self):
        """Calculates the objective value from the output data and checks if it is identical 
        to the solver results."""

        result = sum(
            self.asset_data['ngas_grid']['ngas_balance'] * self.input_data['gas_price'] +
            self.asset_data['chp_1']['co2'] * CO2_PRICE +
            self.asset_data['chp_2']['co2'] * CO2_PRICE +
            self.asset_data['electrical_grid']['power_balance'] * self.input_data['power_price'] +
            self.asset_data['hydrogen_grid']['hydrogen_balance'] * H2_PRICE -
            self.asset_data['heat_grid']['heat_feedin'] * HEAT_PRICE
            )

        result = round(result, 1)
        
        if result == self.objective_value:
            print('\nObjective value and calculated value are the same.')
            print(f'{self.objective_value} == {result}')
        else:
            print('\nObjective value and calculated value are not identical!')
            print(f'{self.objective_value} =/= {result}')


Calculate overview data for assets

In [ ]:
assets = Assets(output_df)

df = pd.DataFrame(assets.postprocess_data)
print(df)

assets.check_objective_value()

Create Plots

In [ ]:
FONTSIZE = 12
LABELSIZE = 11

plt.rc('xtick', labelsize=LABELSIZE)
plt.rc('ytick', labelsize=LABELSIZE)

def plot_heat_storage(xlim=None):
    charge = assets.asset_data['heat_storage']['heat_charging']
    discharge = assets.asset_data['heat_storage']['heat_discharging'] * - 1
    content = assets.asset_data['heat_storage']['heat_content']

    x = range(1, len(charge) + 1)

    fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(17, 7.5), dpi=120)

    axs[0].fill_between(x, content, step='pre', alpha=0.5, label='content', color='gold')
    axs[0].plot(x, content, drawstyle='steps', color='gold')
    axs[1].fill_between(x, discharge, step='pre', alpha=0.5, label='discharge')
    axs[1].fill_between(x, charge, step='pre', alpha=0.5, label='charge')
    axs[1].plot(x, discharge, drawstyle='steps')
    axs[1].plot(x, charge, drawstyle='steps')

    axs[0].set_xlabel('time [h]', fontsize=FONTSIZE)
    axs[0].set_ylabel('heat [MWh]', fontsize=FONTSIZE)
    axs[1].set_xlabel('time [h]', fontsize=FONTSIZE)
    axs[1].set_ylabel('thermal power [MW]')

    axs[0].grid()
    axs[1].grid()

    if xlim:
        axs[0].set_xlim(xlim)
        axs[1].set_xlim(xlim)

    handle_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    handles, labels = [sum(lists, []) for lists in zip(*handle_labels)]

    fig.legend(handles, labels, ncol=3, bbox_to_anchor=(0.99, 1.04))
    fig.tight_layout()


def plot_battery_storage(xlim=None):
    charge = assets.asset_data['battery_storage']['power_charging']
    discharge = assets.asset_data['battery_storage']['power_discharging'] * - 1
    content = assets.asset_data['battery_storage']['power_content']

    x = range(1, len(charge) + 1)

    fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(17, 7.5), dpi=120)

    axs[0].fill_between(x, content, step='pre', alpha=0.5, label='content', color='gold')
    axs[0].plot(x, content, drawstyle='steps', color='gold')
    axs[1].fill_between(x, discharge, step='pre', alpha=0.5, label='discharge')
    axs[1].fill_between(x, charge, step='pre', alpha=0.5, label='charge')
    axs[1].plot(x, discharge, drawstyle='steps')
    axs[1].plot(x, charge, drawstyle='steps')

    axs[0].set_xlabel('time [h]', fontsize=FONTSIZE)
    axs[0].set_ylabel('electrical energy [MWh]', fontsize=FONTSIZE)
    axs[1].set_xlabel('time [h]', fontsize=FONTSIZE)
    axs[1].set_ylabel('power [MW]')

    axs[0].grid()
    axs[1].grid()

    if xlim:
        axs[0].set_xlim(xlim)
        axs[1].set_xlim(xlim)

    handle_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    handles, labels = [sum(lists, []) for lists in zip(*handle_labels)]

    fig.legend(handles, labels, ncol=3, bbox_to_anchor=(0.99, 1.04))
    fig.tight_layout()

def plot_energy_balance(xlim=None):
    # Heat data
    chp_heat = (assets.asset_data['chp_1']['heat']
                + assets.asset_data['chp_2']['heat'])
    hp_heat = assets.asset_data['heatpump']['heat']
    out_heat = assets.asset_data['heat_storage']['heat_discharging']
    in_heat = -assets.asset_data['heat_storage']['heat_charging']
    demand_heat = (assets.asset_data['heat_grid']['heat_feedin']
                   - assets.asset_data['heat_grid']['heat_supply'])
    # Power data
    chp_power = (assets.asset_data['chp_1']['power']
                 + assets.asset_data['chp_2']['power'])
    hp_power = -assets.asset_data['heatpump']['power']
    out_power = assets.asset_data['battery_storage']['power_discharging']
    in_power = -assets.asset_data['battery_storage']['power_charging']
    electrolyzer_power = (-assets.asset_data['electrolyzer_1']['power']
                          - assets.asset_data['electrolyzer_2']['power']
                          - assets.asset_data['electrolyzer_3']['power']
                          - assets.asset_data['electrolyzer_4']['power']
                          - assets.asset_data['electrolyzer_5']['power']
                          - assets.asset_data['electrolyzer_6']['power'])
    pv_power = assets.asset_data['pv']['power']
    balance_power = -assets.asset_data['electrical_grid']['power_balance']
    
    # Hydrogen data
    chp_hydrogen = (-assets.asset_data['chp_1']['hydrogen']
                    - assets.asset_data['chp_2']['hydrogen'])
    electrolyzer_hydrogen = (assets.asset_data['electrolyzer_1']['hydrogen']
                             + assets.asset_data['electrolyzer_2']['hydrogen']
                             + assets.asset_data['electrolyzer_3']['hydrogen']
                             + assets.asset_data['electrolyzer_4']['hydrogen']
                             + assets.asset_data['electrolyzer_5']['hydrogen']
                             + assets.asset_data['electrolyzer_6']['hydrogen'])
    balance_hydrogen = -assets.asset_data['hydrogen_grid']['hydrogen_balance']
    
    x = np.arange(len(chp_heat)) + 1
    
    fig, axs = plt.subplots(nrows=3, ncols=1, dpi=120, figsize=(17, 7.5))
    axs[0].fill_between(x, 
                        chp_heat,
                        step='pre',
                        color='C1',
                        alpha=0.5,
                        label='chp')
    axs[0].fill_between(x,
                        chp_heat, 
                        chp_heat + hp_heat, 
                        step='pre', 
                        color='C3', 
                        alpha=0.5, 
                        label='hp')
    axs[0].fill_between(x,
                        chp_heat + hp_heat,
                        chp_heat + hp_heat + out_heat,
                        step='pre',
                        color='C4',
                        alpha=0.5,
                        label='discharge')
    axs[0].fill_between(x, 
                        in_heat,
                        step='pre',
                        color='C2',
                        alpha=0.5,
                        label='charge')
    axs[0].plot(demand_heat, drawstyle='steps', color='dimgrey', label='demand')
    
    axs[0].set_ylabel('thermal power [MW]')
    axs[0].set_xlabel('time [h]')
    axs[0].grid()
    
    axs[1].fill_between(x, 
                        chp_power, 
                        step='pre',
                        color='C1', 
                        alpha=0.5)
    axs[1].fill_between(x,
                        chp_power,
                        chp_power + pv_power,
                        step='pre',
                        color='C8',
                        alpha=0.5,
                        label='pv')
    axs[1].fill_between(x,
                        electrolyzer_power,
                        step='pre',
                        color='C0',
                        alpha=0.5,
                        label='electrolyzer')
    axs[1].fill_between(x,
                        electrolyzer_power, 
                        electrolyzer_power + hp_power, 
                        step='pre',
                        color='C3', 
                        alpha=0.5)
    axs[1].fill_between(x, 
                        chp_power + pv_power, 
                        chp_power + pv_power + out_power, 
                        step='pre', 
                        color='C4',
                        alpha=0.5)
    axs[1].fill_between(x, 
                        electrolyzer_power + hp_power, 
                        electrolyzer_power + hp_power + in_power, 
                        step='pre', 
                        color='C2',
                        alpha=0.5)
    axs[1].plot(balance_power, 
                drawstyle='steps', 
                color='dimgrey', 
                label='balance')
    
    axs[1].set_ylabel('electrical power [MW]')
    axs[1].set_xlabel('time [h]')
    axs[1].grid()
    
    axs[2].fill_between(x, 
                        chp_hydrogen, 
                        step='pre', 
                        color='C1',
                        alpha=0.5)
    axs[2].fill_between(x, 
                        electrolyzer_hydrogen, 
                        step='pre', 
                        color='C0', 
                        alpha=0.5)
    axs[2].plot(balance_hydrogen, 
                drawstyle='steps', 
                color='dimgrey')
    
    axs[2].set_ylabel('hydrogen [MWh/h]')
    axs[2].set_xlabel('time [h]')
    axs[2].grid()

    if xlim:
        axs[0].set_xlim(xlim)
        axs[1].set_xlim(xlim)
        axs[2].set_xlim(xlim)
    
    fig.legend(ncol=8, bbox_to_anchor=(0.99, 1.04))
    fig.tight_layout()

def plot_costs(xlim=None):
    power_data = assets.asset_data['electrical_grid']['power_balance']
    hydrogen_data = assets.asset_data['hydrogen_grid']['hydrogen_balance']

    ngas_costs = (
        assets.asset_data['ngas_grid']['ngas_balance'] 
        * assets.input_data['gas_price']
        / 1000)
    co2_costs = ((
        assets.asset_data['chp_1']['co2'] * CO2_PRICE 
        + assets.asset_data['chp_2']['co2'] * CO2_PRICE)
        / 1000)
    power_costs = (
        power_data.where(power_data >= 0, 0) 
        * assets.input_data['power_price']
        / 1000)
    power_revenue = (
        power_data.where(power_data <= 0, 0) 
        * assets.input_data['power_price']
        / 1000)
    hydrogen_costs = (
        hydrogen_data.where(hydrogen_data >= 0, 0) 
        * H2_PRICE
        / 1000)
    hydrogen_revenue = (
        hydrogen_data.where(hydrogen_data <= 0, 0) 
        * H2_PRICE 
        / 1000)

    costs = (
        assets.asset_data['ngas_grid']['ngas_balance'] * assets.input_data['gas_price'] +
        assets.asset_data['chp_1']['co2'] * CO2_PRICE +
        assets.asset_data['chp_2']['co2'] * CO2_PRICE +
        assets.asset_data['electrical_grid']['power_balance'] * assets.input_data['power_price'] +
        assets.asset_data['hydrogen_grid']['hydrogen_balance'] * H2_PRICE
    ) / 1000

    x = range(1, len(ngas_costs) + 1)

    fig, ax = plt.subplots(figsize=(17, 7.5), dpi=120)
    ax.fill_between(x,
                    ngas_costs,
                    step='pre',
                    alpha=0.5,
                    label='ngas')
    ax.fill_between(x,
                    ngas_costs,
                    ngas_costs+ co2_costs,
                    step='pre',
                    alpha=0.5,
                    label='co2')
    ax.fill_between(x,
                    ngas_costs + co2_costs,
                    ngas_costs + co2_costs + power_costs,
                    step='pre',
                    alpha=0.5,
                    label='power_costs')
    ax.fill_between(x,
                    ngas_costs + co2_costs + power_costs,
                    ngas_costs + co2_costs + power_costs + hydrogen_costs,
                    step='pre',
                    alpha=0.5,
                    label='hydrogen_costs')
    ax.fill_between(x,
                    power_revenue,
                    step='pre',
                    alpha=0.5,
                    label='power_revenue')
    ax.fill_between(x,
                    power_revenue,
                    power_revenue + hydrogen_revenue,
                    step='pre',
                    alpha=0.5,
                    label='hydrogen_revenue')
    ax.plot(x, costs, drawstyle='steps', label='balance')
    ax.set_xlabel('t [h]')
    ax.set_ylabel('costs [T EUR]')
    if xlim:
        ax.set_xlim(xlim)
    ax.grid()
    ax.legend(ncol=7, bbox_to_anchor=(0.99, 1.07))
    fig.tight_layout()

    fig, ax = plt.subplots(figsize=(17, 7.5), dpi=120)
    ax.fill_between(x, 
                    ngas_costs.cumsum(), 
                    step='pre', 
                    alpha=0.5,
                    label='gas_costs')
    ax.fill_between(x, 
                    ngas_costs.cumsum(), 
                    ngas_costs.cumsum() + co2_costs.cumsum(), 
                    step='pre', 
                    alpha=0.5,
                    label='co2_costs')
    ax.fill_between(x, 
                    ngas_costs.cumsum() + co2_costs.cumsum(), 
                    ngas_costs.cumsum() + co2_costs.cumsum() + power_costs.cumsum(),
                    step='pre', 
                    alpha=0.5,
                    label='power_costs')
    ax.fill_between(x,  
                    ngas_costs.cumsum() + co2_costs.cumsum() + power_costs.cumsum(),
                    ngas_costs.cumsum() + co2_costs.cumsum() + power_costs.cumsum() + hydrogen_costs.cumsum(),
                    step='pre', 
                    alpha=0.5,
                    label='hydrogen_costs')
    ax.fill_between(x,  
                    power_revenue.cumsum(),
                    step='pre', 
                    alpha=0.5,
                    label='power_revenue')
    ax.fill_between(x,  
                    power_revenue.cumsum(),
                    power_revenue.cumsum() + hydrogen_revenue.cumsum(),
                    step='pre', 
                    alpha=0.5,
                    label='hydrogen_revenue')
    ax.plot(x, costs.cumsum(), drawstyle='steps', label='balance')
    ax.set_xlabel('t [h]')
    ax.set_ylabel('costs [T EUR]')
    if xlim:
        ax.set_xlim(xlim)
    ax.grid()
    ax.legend(ncol=7, bbox_to_anchor=(0.99, 1.07))
    fig.tight_layout()

In [ ]:
XLIM = (0, 1000)   # Needs to be None or a tuple of min and max xlim

plot_battery_storage(xlim=XLIM)
plot_heat_storage(xlim=XLIM)
plot_energy_balance(xlim=XLIM)
plot_costs(xlim=XLIM)

In [ ]:
def calc_costs(assets):
    power_data = assets.asset_data['electrical_grid']['power_balance']
    hydrogen_data = assets.asset_data['hydrogen_grid']['hydrogen_balance']
    ngas_costs = (
        assets.asset_data['ngas_grid']['ngas_balance'] 
        * assets.input_data['gas_price']
        / 1000).sum()
    co2_costs = ((
        assets.asset_data['chp_1']['co2'] * CO2_PRICE 
        + assets.asset_data['chp_2']['co2'] * CO2_PRICE)
        / 1000).sum()
    power_costs = (
        power_data.where(power_data >= 0, 0) 
        * assets.input_data['power_price']
        / 1000).sum()
    power_revenue = (
        power_data.where(power_data <= 0, 0) 
        * assets.input_data['power_price']
        / 1000).sum()
    hydrogen_costs = (
        hydrogen_data.where(hydrogen_data >= 0, 0) 
        * H2_PRICE
        / 1000).sum()
    hydrogen_revenue = (
        hydrogen_data.where(hydrogen_data <= 0, 0) 
        * H2_PRICE 
        / 1000).sum()
    df = pd.DataFrame({
        'ngas_costs' : ngas_costs,
        'co2_costs' : co2_costs,
        'power_costs' : power_costs,
        'power_revenue' : power_revenue,
        'hydrogen_costs' : hydrogen_costs,
        'hydrogen_revenue' : hydrogen_revenue
    }.items(), columns=['position', 'value'])
    df.set_index('position', inplace=True)

    return df


In [ ]:
output_df_s1 = pd.read_csv(OUTPUT_PATH + 's1_output.csv', index_col=0)
output_df_s2 = pd.read_csv(OUTPUT_PATH + 's2_output.csv', index_col=0)
output_df_s3 = pd.read_csv(OUTPUT_PATH + 's3_output.csv', index_col=0)

In [ ]:
assets_s1 = Assets(output_df_s1)
assets_s2 = Assets(output_df_s2)
assets_s3 = Assets(output_df_s3)

df_s1 = pd.DataFrame(assets_s1.postprocess_data)
df_s2 = pd.DataFrame(assets_s2.postprocess_data)
df_s3 = pd.DataFrame(assets_s3.postprocess_data)

cost_df_s1 = calc_costs(assets_s1)
cost_df_s2 = calc_costs(assets_s2)
cost_df_s3 = calc_costs(assets_s3)

In [ ]:
scenarios = ['s1', 's2', 's3']
thermal_assets = {}

for thermal_asset in ['chp_1', 'chp_2', 'heatpump']:
    data_list = []
    for df in [df_s1, df_s2, df_s3]:
        data_list.append(round(df[thermal_asset]['heat'] / 1000, 1))
    thermal_assets[thermal_asset] = data_list

x = np.arange(len(scenarios))
width = 0.25
mulitplier = 0

fig, ax = plt.subplots(dpi=120)

for asset, data in thermal_assets.items():
    offset = width * mulitplier
    bars = ax.bar(x + offset, data, width, label=asset)
    ax.bar_label(bars, padding=4)
    mulitplier += 1
    
ax.set_xticks(x + width, scenarios)
ax.legend(ncol=3, bbox_to_anchor=(0.99, 1.14))
ax.set_ylim(0, 300)
ax.set_ylabel('annual produced heat [GWh]')
fig.tight_layout()


In [ ]:
cost_df_s1

In [ ]:
scenarios = ['s1', 's2', 's3']
cost_data = {}

for cost_position in ['ngas_costs', 'co2_costs', 'power_costs', 'power_revenue', 'hydrogen_costs', 'hydrogen_revenue']:
    data_list = []
    for df in [cost_df_s1, cost_df_s2, cost_df_s3]:
        data_list.append(round(df['value'][cost_position], 1))
    cost_data[cost_position] = data_list

x = np.arange(len(scenarios))
width = 0.15
mulitplier = 0

fig, ax = plt.subplots(figsize=(17, 7.5), dpi=120)

for position, data in cost_data.items():
    offset = width * mulitplier
    bars = ax.bar(x + offset, data, width, label=position)
    ax.bar_label(bars, padding=4)
    mulitplier += 1
    
ax.set_xticks(x + width, scenarios)
ax.legend(ncol=6, bbox_to_anchor=(0.99, 1.06))
ax.set_ylim(-100000, 100000)
ax.set_ylabel('annual costs/revenue [TEUR]')
fig.tight_layout()